In [147]:
import numpy as np
import scipy.integrate as integrate

In [148]:
f = lambda x: 4*np.cos(0.5*x)*np.exp(-5/4*x) + 2*np.sin(4.5*x)*np.exp(x/8) + 2

In [149]:
a = 1.3; b = 2.2; alpha = 0; beta = 5/6

In [150]:
p = lambda x, j: (x-a)**(-alpha) * (b-x)**(-beta)*x**j

### 1. Задаем узлы

In [151]:
x_n = np.array([[a], [(a+b)/2], [b]]); x_n

array([[1.3 ],
       [1.75],
       [2.2 ]])

### 2. Вычисляем моменты весовой функции.

In [152]:
m = []
for j in range(3):
    m.append(integrate.quad(p, a, b, args = (j)))
m = np.array(m)
M = np.array(m[:, 0]); M

array([ 5.89555916, 12.2122297 , 25.56664299])

In [153]:
X = np.array([ [1,    1,    1], 
               [x_n[0]**1, x_n[1]**1, x_n[2]**1], 
               [x_n[0]**2, x_n[1]**2, x_n[2]**2] ], dtype=np.float64); X

array([[1.    , 1.    , 1.    ],
       [1.3   , 1.75  , 2.2   ],
       [1.69  , 3.0625, 4.84  ]])

### Решаем СЛАУ

In [154]:
A = np.linalg.solve(X, M); A

array([0.06478634, 1.55487279, 4.27590003])

### Вычисляем интеграл.

In [155]:
integral = np.sum([A[j]*f(x_n[j]) for j in range(3)]); integral

11.429030895719116

Смотрим на то, что даёт встроенное интегрирование (ответ тот же, что и через Wolfram

In [156]:
f_p = lambda x: (4*np.cos(0.5*x)*np.exp(-5/4*x) + 2*np.sin(4.5*x)*np.exp(x/8) + 2) * (b-x)**(-beta)

In [157]:
integrate.quad(f_p, a, b)[0]

10.83954513007571

In [158]:
integral - integrate.quad(f_p, a, b)[0]

0.5894857656434063

## Вычислим методическую погрешность

### Найдем M3

In [168]:
import sympy as sp

In [169]:
y = sp.Symbol('y')

In [170]:
f = 4*sp.cos(0.5*y)*sp.exp(-5/4*y) + 2*sp.sin(4.5*y)*sp.exp(y/8) + 2

In [171]:
df = sp.diff(f, y, 3)

In [172]:
df

-(15.18359375*exp(y/8)*sin(4.5*y) + 181.828125*exp(y/8)*cos(4.5*y) + 8.875*exp(-1.25*y)*sin(0.5*y) + 4.0625*exp(-1.25*y)*cos(0.5*y))

In [173]:
y_area = np.linspace(a, b, 2000)
val = []
for i in y_area:
    val.append(df.subs(y,i))
M3 = max(abs(max(val)), abs(min(val)))
M3

237.018194528471

### Вычислим оценку методической погрешности

In [174]:
p_omega = lambda x: abs( (x-x_n[0])*(x-x_n[1])*(x-x_n[2])*(b-x)**(-beta) )

In [175]:
R = M3/6 * integrate.quad(p_omega, a, b)[0]

In [176]:
R

2.59497030395034